# Setup

In [1]:
import pandas as pd
import numpy as np
import nba_api
import json

## nba_api packages

In [44]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
from nba_api.stats.static.players import get_active_players
from nba_api.stats.endpoints import CommonTeamRoster

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import LeagueDashTeamShotLocations
from nba_api.stats.endpoints import ShotChartLineupDetail
from nba_api.stats.endpoints import ShotChartDetail

## Get spurs test game

In [4]:
# Get Spurs' team info
all_teams = get_teams()
spurs_info = find_teams_by_nickname('spurs') # find spurs nickname
spurs_id = spurs_info[0]['id'] # get spurs team ID

# find all Spurs games
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
spurs_games_df = gamefinder.get_data_frames()[0]
spurs_games_df.head(n=10)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612759,SAS,San Antonio Spurs,0021901314,2020-08-13,SAS @ UTA,L,239,112,...,0.667,9,35,44,24,4,8,15,20,-6.0
1,22019,1610612759,SAS,San Antonio Spurs,0021901298,2020-08-11,SAS vs. HOU,W,240,123,...,0.900,9,50,59,28,10,5,22,20,18.0
2,22019,1610612759,SAS,San Antonio Spurs,0021901287,2020-08-09,SAS @ NOP,W,241,122,...,0.969,15,36,51,22,12,6,17,27,9.0
3,22019,1610612759,SAS,San Antonio Spurs,0021901274,2020-08-07,SAS vs. UTA,W,240,119,...,0.769,8,39,47,27,10,5,13,18,8.0
4,22019,1610612759,SAS,San Antonio Spurs,0021901264,2020-08-05,SAS vs. DEN,L,239,126,...,0.852,10,25,35,29,10,5,7,20,-6.0
5,22019,1610612759,SAS,San Antonio Spurs,0021901254,2020-08-03,SAS @ PHI,L,240,130,...,0.897,13,31,44,21,7,6,14,23,-2.0
6,22019,1610612759,SAS,San Antonio Spurs,0021901246,2020-08-02,SAS @ MEM,W,241,108,...,0.727,12,37,49,22,5,6,16,20,2.0
7,22019,1610612759,SAS,San Antonio Spurs,0021901237,2020-07-31,SAS vs. SAC,W,240,129,...,0.815,9,33,42,32,6,4,15,17,9.0
8,12019,1610612759,SAS,San Antonio Spurs,0011900130,2020-07-28,SAS @ IND,W,241,118,...,0.905,7,41,48,29,8,8,12,18,7.0
9,12019,1610612759,SAS,San Antonio Spurs,0011900115,2020-07-25,SAS vs. BKN,L,240,119,...,0.857,12,45,57,21,4,11,24,32,-5.0


# Test package outputs

In [20]:
shots = LeagueDashTeamShotLocations(
    distance_range= 'By Zone',
    season = '2019-20',
    last_n_games=1
)

In [21]:
shots_json = shots.data_sets[0].get_dict()

In [41]:
shot_loc = ShotChartLineupDetail(
    team_id_nullable=spurs_id,
    #game_id_nullable=spurs_games_df.iloc[0].GAME_ID,
    season='2019-20'
)

In [42]:
shot_loc.data_sets[0].get_data_frame()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,GROUP_ID,GROUP_NAME,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM


In [45]:
players = get_active_players()

In [55]:
steven_adams_id = players[0]['id']

okc_id = find_teams_by_nickname('thunder')[0]['id']

In [56]:
adams_shots = ShotChartDetail(
    player_id=steven_adams_id,
    season_nullable='2019-20',
    team_id=okc_id
)

In [61]:
# Lonnie Walker shots from the very last game

spurs_shots = ShotChartDetail(
    player_id= 1629022,
    team_id=spurs_id,
    game_id_nullable=spurs_games_df.iloc[0].GAME_ID,
)

In [62]:
spurs_shots.data_sets[0].get_data_frame()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021901314,44,1629022,Lonnie Walker IV,1610612759,San Antonio Spurs,1,8,34,...,Center(C),8-16 ft.,10,-43,99,1,1,20200813,UTA,SAS
1,Shot Chart Detail,0021901314,276,1629022,Lonnie Walker IV,1610612759,San Antonio Spurs,2,4,44,...,Left Side Center(LC),24+ ft.,24,-149,190,1,1,20200813,UTA,SAS
2,Shot Chart Detail,0021901314,356,1629022,Lonnie Walker IV,1610612759,San Antonio Spurs,3,11,31,...,Center(C),Less Than 8 ft.,2,-19,12,1,1,20200813,UTA,SAS
3,Shot Chart Detail,0021901314,462,1629022,Lonnie Walker IV,1610612759,San Antonio Spurs,3,3,38,...,Center(C),Less Than 8 ft.,3,4,32,1,1,20200813,UTA,SAS
4,Shot Chart Detail,0021901314,557,1629022,Lonnie Walker IV,1610612759,San Antonio Spurs,4,7,46,...,Right Side Center(RC),24+ ft.,25,133,217,1,1,20200813,UTA,SAS
5,Shot Chart Detail,0021901314,565,1629022,Lonnie Walker IV,1610612759,San Antonio Spurs,4,6,53,...,Center(C),24+ ft.,26,13,262,1,1,20200813,UTA,SAS


In [63]:
adams_shots_df = adams_shots.data_sets[0].get_data_frame()
adams_shots_df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021900011,550,203500,Steven Adams,1610612760,Oklahoma City Thunder,4,9,10,...,Center(C),Less Than 8 ft.,2,-23,6,1,1,20191023,UTA,OKC
1,Shot Chart Detail,0021900022,7,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,11,42,...,Center(C),Less Than 8 ft.,0,-3,1,1,1,20191025,OKC,WAS
2,Shot Chart Detail,0021900022,283,203500,Steven Adams,1610612760,Oklahoma City Thunder,2,3,45,...,Center(C),Less Than 8 ft.,1,16,1,1,1,20191025,OKC,WAS
3,Shot Chart Detail,0021900022,394,203500,Steven Adams,1610612760,Oklahoma City Thunder,3,7,32,...,Center(C),Less Than 8 ft.,1,13,5,1,1,20191025,OKC,WAS
4,Shot Chart Detail,0021900036,58,203500,Steven Adams,1610612760,Oklahoma City Thunder,1,6,42,...,Center(C),Less Than 8 ft.,2,-25,4,1,1,20191027,OKC,GSW


In [64]:
adams_shots_df.SHOT_ZONE_AREA.unique()

array(['Center(C)', 'Right Side(R)', 'Left Side(L)', 'Back Court(BC)',
       'Right Side Center(RC)'], dtype=object)

In [65]:
adams_shots_df.SHOT_ZONE_RANGE.unique()

array(['Less Than 8 ft.', '8-16 ft.', 'Back Court Shot', '16-24 ft.'],
      dtype=object)

In [ ]:
# get rosters of each team
# grab separate shot charts for a game
# separate by sections 